## Imports

In [15]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
import sys

## Data Loading

Load the data using read_json

In [2]:
df = pd.read_json("echr_2_0_0_unstructured_cases.json")
df

,__articles,__conclusion,_decision_body,applicability,application,appno,article,attachments,conclusion,content,...,originatingbody_name,originatingbody_type,paragraphs,parties,rank,representedby,respondent,scl,separateopinion,typedescription
0,13;13+P1-1;P1-1,Violation of Article 1 of Protocol No. 1 - Pro...,"Françoise Tulkens, President,\n\tVladimiro Zag...",,MS WORD,33888/05,"[p1-1, 13]",{'001-95845.docx': {}},"[{'article': 'p1-1', 'base_article': 'p1-1', '...","{'001-95845.docx': [{'content': 'PROCEDURE', '...",...,Second Section,Court,"[P1-1, 13]","[POPOVIC, SERBIA]",134.36655,[STOJANOVIC S.],SRB,[],FALSE,15
1,10;10-1;10-2;35;35-3-b;41,Preliminary objection dismissed (Art. 35) Admi...,"Tim Eicke, President,\n\tYonko Grozev,\n\tFari...",,MS WORD,10783/14,"[35, 41, 10]",{'001-209033.docx': {}},"[{'details': ['Art. 35'], 'element': 'Prelimin...",{'001-209033.docx': [{'content': 'INTRODUCTION...,...,Fourth Section,Court,"[41, 35, 35-3-b, 10-2, 10-1, 10]","[HANDZHIYSKI, BULGARIA]",138.37206,"[EKIMDZHIEV M., BONCHEVA K.]",BGR,"[Albert-Engelmann-Gesellschaft mbH v. Austria,...",TRUE,15
2,5;5-4;8;8-1;8-2;35;35-1;35-3-a;37;37-1-c;41,Preliminary objections dismissed (Art. 35) Adm...,"Helena Jäderblom, President,\n\tBranko Lubarda...",,MS WORD,29431/05;7070/06;5402/07,"[41, 8, 5, 35, 37]",{'001-178343.docx': {'table-0': [{'Applicant’s...,"[{'details': ['Art. 35'], 'element': 'Prelimin...","{'001-178343.docx': [{'content': 'PROCEDURE', ...",...,Third Section,Court,"[37-1-c, 41, 8, 8-2, 5, 35, 37, 8-1, 35-3-a, 3...","[ZUBKOV AND OTHERS, RUSSIA]",134.47542,"[KULIK N., OVCHINNIKOV M., MIKHAYLOV A.]",RUS,"[Ahtinen v. Finland (dec.), no. 48907/99, 31 M...",TRUE,15
3,6;6-1,Violation of Art. 6-1,"Nicolas Bratza, President,\n\tJosep Casadevall...",,MS WORD,11101/04,[6],{'001-84572.docx': {}},"[{'article': '6-1', 'base_article': '6', 'elem...","{'001-84572.docx': [{'content': 'PROCEDURE', '...",...,Fourth Section,Court,"[6-1, 6]","[RYGALSKI, POLAND]",134.29863,[N/A],POL,[],FALSE,15
4,3;8;8-1,Violation of Article 3 - Prohibition of tortur...,"Nina Vajić, President,\n\tAnatoly Kovler,\n\tP...",,MS WORD,19433/07,"[3, 8]",{'001-112534.docx': {}},"[{'article': '3', 'base_article': '3', 'detail...","{'001-112534.docx': [{'content': 'PROCEDURE', ...",...,First Section,Court,"[3, 8, 8-1]","[TYAGUNOVA, RUSSIA]",132.75772,[LOBOV A.],RUS,[],FALSE,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16091,3,Violation of Art. 3 (substantive aspect),"Josep Casadevall, President,\n\tCorneliu Bîrsa...",,MS WORD,31725/04,[3],{'001-106551.docx': {}},"[{'article': '3', 'base_article': '3', 'detail...","{'001-106551.docx': [{'content': 'PROCEDURE', ...",...,Third Section,Court,[3],"[BADILA, ROMANIA]",132.52425,[N/A],ROU,[],FALSE,15
16092,5;5-1;5-4;13+3;13;3,Violation of Art. 5-1;Violation of Art. 5-4;Vi...,"Christos Rozakis, President,\n\tNina Vajić,\n\...",,MS WORD,54219/08,"[3, 5, 13]",{'001-100198.docx': {}},"[{'article': '5-1', 'base_article': '5', 'elem...","{'001-100198.docx': [{'content': 'PROCEDURE', ...",...,First Section,Court,"[3, 5, 5-1, 13, 5-4]","[KARIMOV, RUSSIA]",134.37077,"[RYABININA E., ZILBERMAN R.]",RUS,[],FALSE,15
16093,6,Violation of Article 6 - Right to a fair trial,"Nicolas Bratza, President,\n\tLech Garlicki,\n...",,MS WORD,25924/06,[6],{'001-95191.docx': {}},"[{'article': '6', 'base_article': '6', 'elemen...","{'001-95191.docx': [{'content': 'PROCEDURE', '...",...,Fourth Section,Court,[6],"[NOWINSKI, POLAND]",134.44272,[BODNAR A.],POL,[],FALSE,15
16094,3,Violation of Article 3 - Prohibition of tortur...,"Josep Casadevall, President,\n\tAlvina Gyulumy...",,MS WORD,9443/10,[3],{'001-147270.docx': {}},"[{'article': '3', 'base_article': '3', 'detail...","{'001-147270.docx': [{'content': 'PROCEDURE', ...",...,Third Section,Court,[3],"[MARIAN CHIRIȚĂ, ROMANIA]",133.14912,[],ROU,[],FALSE,15


Let's observe the columns

In [3]:
df.columns

Index(['__articles', '__conclusion', '_decision_body', 'applicability',
       'application', 'appno', 'article', 'attachments', 'conclusion',
       'content', 'country', 'decision_body', 'decisiondate', 'docname',
       'doctypebranch', 'documentcollectionid', 'documents', 'ecli',
       'externalsources', 'extractedappno', 'importance', 'introductiondate',
       'issue', 'itemid', 'judgementdate', 'kpdate', 'kpthesaurus',
       'languageisocode', 'originatingbody', 'originatingbody_name',
       'originatingbody_type', 'paragraphs', 'parties', 'rank',
       'representedby', 'respondent', 'scl', 'separateopinion',
       'typedescription'],
      dtype='object')

Recursive function to extract a section of text from the json file

In [4]:
def extract_section_markdown(case_data, section_name):
    section_text = ""
    if 'content' in case_data:
        documents = case_data['content']
        for key, sections in documents.items():
            for section in sections:
                if section.get('section_name', '').upper() == section_name.upper():
                    # Append the section title with a heading format
                    section_text += f"### {section['content']}\n\n"
                    section_text += extract_elements(section['elements'])
                    section_text += "\n"

    return section_text.strip()

def extract_elements(elements, indent=0):
    content_text = ""
    for element in elements:
        # Add indentation for each level of nesting
        prefix = "  " * indent
        content_text += f"{prefix}- {element['content']}\n"
        if 'elements' in element:
            # Recursively add further nested elements with increased indentation
            content_text += extract_elements(element['elements'], indent + 1)
    return content_text

Extract the law section

In [5]:
df['law'] = df.apply(lambda x: extract_section_markdown(x, 'law'), axis = 1)


In [6]:
print(df['law'][1])

### THE LAW

- ALLEGED VIOLATION OF ARTICLE 10 OF THE CONVENTION
  - 26.  The applicant complained that the interference with his right to freedom of expression had not been necessary in a democratic society. He relied on Article 10 of the Convention, which provides, in so far as relevant:
  - “1.  Everyone has the right to freedom of expression. This right shall include freedom to hold opinions and to receive and impart information and ideas without interference by public authority ...
  - 2.  The exercise of these freedoms, since it carries with it duties and responsibilities, may be subject to such formalities, conditions, restrictions or penalties as are prescribed by law and are necessary in a democratic society, in the interests of national security, territorial integrity or public safety, for the prevention of disorder or crime, for the protection of health or morals, for the protection of the reputation or rights of others, for preventing the disclosure of information received 

Extract the facts section

In [7]:
df['facts'] = df.apply(lambda x: extract_section_markdown(x, 'facts'), axis = 1)


In [8]:
print(df['facts'][1])

### THE FACTS

- 2.  The applicant was born in 1971 and lives in Blagoevgrad. He was represented by Mr M. Ekimdzhiev and Ms K. Boncheva, lawyers practising in Plovdiv.
- 3.  The Government were represented by their Agent, Ms I. Stancheva-Chinova of the Ministry of Justice.
- Background to the case
- Background to the case
  - 4.  Following parliamentary elections on 12 May 2013, on 29 May 2013 a new government was formed, led by Mr Plamen Oresharski and supported in Parliament by the Coalition for Bulgaria (whose main member was the Bulgarian Socialist Party – see paragraph 8 below), which had eighty-four members of Parliament, the Movement for Rights and Freedoms, which had thirty-six members of Parliament, and the chairman of another political party, Ataka, Mr Volen Siderov, who was also a member of Parliament. Together, these provided a majority of one hundred twenty-one out of the total of two hundred and forty members of Parliament.
  - 4.  Following parliamentary elections on 12 

Extract the conclusion section

In [9]:
df['the_conclusion'] = df.apply(lambda x: extract_section_markdown(x, 'conclusion'), axis = 1)


In [10]:
print(df['the_conclusion'][1])

### FOR THESE REASONS, THE COURT

- Declares, unanimously, the application admissible;
- Holds, by six votes to one, that there has been a violation of Article 10 of the Convention;
- Holds, by six votes to one,
- that the respondent State is to pay the applicant, within three months from the date on which the judgment becomes final in accordance with Article 44 § 2 of the Convention, the following amounts, to be converted into the currency of the respondent State at the rate applicable at the date of settlement:
- that from the expiry of the above-mentioned three months until settlement simple interest shall be payable on the above amounts at a rate equal to the marginal lending rate of the European Central Bank during the default period plus three percentage points;
- Dismisses, unanimously, the remainder of the applicant’s claim for just satisfaction.
- Done in English, and notified in writing on 6 April 2021, pursuant to Rule 77 §§ 2 and 3 of the Rules of Court.
- In accordance wit

Function that cleans up html text to make it readable

In [ ]:
def get_full_text_from_html(html_text):
    soup = BeautifulSoup(html_text, "html.parser")
    # Remove script and style elements
    for script_or_style in soup(["script", "style"]):
        script_or_style.decompose()

    text = soup.get_text()
    
    # Replace multiple whitespace characters with a single space
    text = ' '.join(text.split())

    # Replace non-breaking space characters with regular spaces
    text = text.replace(u'\xa0', u' ')
        
    return text


Functions that requests the webpage in html format and uses the above function to clean up the text

In [ ]:
BASE_URL = 'https://hudoc.echr.coe.int/app/conversion/docx/html/body?library=ECHR&id='

def full_text_extraction(dataframe):
    text_list = []
    for itemid in dataframe['itemid']:
        r = requests.get(BASE_URL + itemid, timeout=5)
        text_list.append(get_full_text_from_html(r.text))
    return text_list


If you run the below line it will take around 1.5 hours to run, so you might want to avoid that

In [ ]:
text_list = full_text_extraction(df)

What I would suggest instead is that you load the data from local memory

In [23]:
filename = "text_list.csv"
text_list= []

# Set the maximum field size to a higher limit, e.g., the maximum integer size
csv.field_size_limit(sys.maxsize)

with open(filename, 'r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    for row in reader:
        if row:  # This checks if the row is not empty
            full_text = ' '.join(row)
            text_list.append(full_text)  



Save it in a column in the dataframe

In [25]:
df['full_text'] = text_list

Extract the judgement type

In [26]:
df['judgement_type_1'] = df['documentcollectionid'].apply(lambda x: x[1])
df['judgement_type_2'] = df['documentcollectionid'].apply(lambda x: x[2])


In [27]:
df

,__articles,__conclusion,_decision_body,applicability,application,appno,article,attachments,conclusion,content,...,respondent,scl,separateopinion,typedescription,law,facts,the_conclusion,full_text,judgement_type_1,judgement_type_2
0,13;13+P1-1;P1-1,Violation of Article 1 of Protocol No. 1 - Pro...,"Françoise Tulkens, President,\n\tVladimiro Zag...",,MS WORD,33888/05,"[p1-1, 13]",{'001-95845.docx': {}},"[{'article': 'p1-1', 'base_article': 'p1-1', '...","{'001-95845.docx': [{'content': 'PROCEDURE', '...",...,SRB,[],FALSE,15,### THE LAW\n\n- I. THE APPLICANT'S DEATH\n ...,### THE FACTS\n\n- I. THE CIRCUMSTANCES OF TH...,"### FOR THESE REASONS, THE COURT UNANIMOUSLY\n...",SECOND SECTION CASE OF МILICA POPOVIĆ v. SERBI...,JUDGMENTS,CHAMBER
1,10;10-1;10-2;35;35-3-b;41,Preliminary objection dismissed (Art. 35) Admi...,"Tim Eicke, President,\n\tYonko Grozev,\n\tFari...",,MS WORD,10783/14,"[35, 41, 10]",{'001-209033.docx': {}},"[{'details': ['Art. 35'], 'element': 'Prelimin...",{'001-209033.docx': [{'content': 'INTRODUCTION...,...,BGR,"[Albert-Engelmann-Gesellschaft mbH v. Austria,...",TRUE,15,### THE LAW\n\n- ALLEGED VIOLATION OF ARTICLE ...,### THE FACTS\n\n- 2. The applicant was born ...,"### FOR THESE REASONS, THE COURT\n\n- Declares...",FOURTH SECTIONCASE OF HANDZHIYSKI v. BULGARIA(...,JUDGMENTS,CHAMBER
2,5;5-4;8;8-1;8-2;35;35-1;35-3-a;37;37-1-c;41,Preliminary objections dismissed (Art. 35) Adm...,"Helena Jäderblom, President,\n\tBranko Lubarda...",,MS WORD,29431/05;7070/06;5402/07,"[41, 8, 5, 35, 37]",{'001-178343.docx': {'table-0': [{'Applicant’s...,"[{'details': ['Art. 35'], 'element': 'Prelimin...","{'001-178343.docx': [{'content': 'PROCEDURE', ...",...,RUS,"[Ahtinen v. Finland (dec.), no. 48907/99, 31 M...",TRUE,15,### THE LAW\n\n- I. JOINDER OF THE APPLICATIO...,### THE FACTS\n\n- I. THE CIRCUMSTANCES OF TH...,"### FOR THESE REASONS, THE COURT, UNANIMOUSLY,...",THIRD SECTION CASE OF ZUBKOV AND OTHERS v. RUS...,JUDGMENTS,CHAMBER
3,6;6-1,Violation of Art. 6-1,"Nicolas Bratza, President,\n\tJosep Casadevall...",,MS WORD,11101/04,[6],{'001-84572.docx': {}},"[{'article': '6-1', 'base_article': '6', 'elem...","{'001-84572.docx': [{'content': 'PROCEDURE', '...",...,POL,[],FALSE,15,### THE LAW\n\n- I. ALLEGED VIOLATION OF ARTI...,### THE FACTS\n\n- I. THE CIRCUMSTANCES OF TH...,"### FOR THESE REASONS, THE COURT UNANIMOUSLY\n...",FOURTH SECTION CASE OF RYGALSKI v. POLAND (App...,JUDGMENTS,CHAMBER
4,3;8;8-1,Violation of Article 3 - Prohibition of tortur...,"Nina Vajić, President,\n\tAnatoly Kovler,\n\tP...",,MS WORD,19433/07,"[3, 8]",{'001-112534.docx': {}},"[{'article': '3', 'base_article': '3', 'detail...","{'001-112534.docx': [{'content': 'PROCEDURE', ...",...,RUS,[],FALSE,15,### THE LAW\n\n- I. ALLEGED VIOLATION OF ARTI...,### THE FACTS\n\n- I. THE CIRCUMSTANCES OF TH...,"### FOR THESE REASONS, THE COURT UNANIMOUSLY\n...",FIRST SECTION CASE OF TYAGUNOVA v. RUSSIA (App...,JUDGMENTS,CHAMBER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16091,3,Violation of Art. 3 (substantive aspect),"Josep Casadevall, President,\n\tCorneliu Bîrsa...",,MS WORD,31725/04,[3],{'001-106551.docx': {}},"[{'article': '3', 'base_article': '3', 'detail...","{'001-106551.docx': [{'content': 'PROCEDURE', ...",...,ROU,[],FALSE,15,### THE LAW\n\n- I. ALLEGED VIOLATION OF ARTI...,### THE FACTS\n\n- I. THE CIRCUMSTANCES OF TH...,"### FOR THESE REASONS, THE COURT UNANIMOUSLY\n...",THIRD SECTION CASE OF BĂDILĂ v. ROMANIA (Appli...,JUDGMENTS,CHAMBER
16092,5;5-1;5-4;13+3;13;3,Violation of Art. 5-1;Violation of Art. 5-4;Vi...,"Christos Rozakis, President,\n\tNina Vajić,\n\...",,MS WORD,54219/08,"[3, 5, 13]",{'001-100198.docx': {}},"[{'article': '5-1', 'base_article': '5', 'elem...","{'001-100198.docx': [{'content': 'PROCEDURE', ...",...,RUS,[],FALSE,15,### THE LAW\n\n- I. THE GOVERNMENT'S PRELIMIN...,### THE FACTS\n\n- I. THE CIRCUMSTANCES OF TH...,"### FOR THESE REASONS, THE COURT UNANIMOUSLY\n...",FIRST SECTION CASE OF KARIM

In [ ]:
echr_df = df[['itemid', 'docname','article','appno','judgementdate', 'law', 'facts', 'the_conclusion', 'full_text']]